In [38]:
import numpy as np
import glob2 as glob
import pandas as pd
import copy
from tqdm import tqdm
import torch
from sklearn.preprocessing import StandardScaler

In [2]:
files = glob.glob('16_8_model_weights_3_layers/*.npy')

weights_dict = {'bias' : {}, 'weight' : {}}

for file in files:
    layer_name = file.split('/')[-1].split('.')[0]
    if layer_name.split('_')[-1] == 'bias':
        weights_dict['bias'].update({layer_name.split('_')[0]: np.load(file)})
    else:
        weights_dict['weight'].update({layer_name.split('_')[0]: np.load(file)})

scaler = StandardScaler()


test_df = pd.read_csv('Iris.csv').drop(columns = ['Id'])
test_df['Species'] = test_df['Species'].map({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2})

test_y = test_df['Species']
test_x = test_df.drop(columns = ['Species'])
test_x = pd.DataFrame(scaler.fit_transform(test_x), columns = test_x.columns)

order= ['fc1', 'fc2', 'fc3']

In [105]:
def choose_neurons_numeric(chosen_neurons, _weights_dict):
    
    weights_dict_ = copy.deepcopy(_weights_dict)
        
    weights_dict_['weight']['fc1'] = weights_dict_['weight']['fc1'][chosen_neurons[0], :]
    weights_dict_['bias']['fc1'] = weights_dict_['bias']['fc1'][chosen_neurons[0]]

    weights_dict_['weight']['fc2'] = weights_dict_['weight']['fc2'][chosen_neurons[1], :][:, chosen_neurons[0]]
    weights_dict_['bias']['fc2'] = weights_dict_['bias']['fc2'][chosen_neurons[1]]

    weights_dict_['weight']['fc3'] = weights_dict_['weight']['fc3'][: ,chosen_neurons[1]]
    
    return weights_dict_

def choose_neurons_matmul(chosen_neurons, _weights_dict):
    weights_dict_ = copy.deepcopy(_weights_dict)

    weights_dict_['weight']['fc1'] = chosen_neurons[0] * weights_dict_['weight']['fc1']

    weights_dict_['weight']['fc2'] = chosen_neurons[1] * weights_dict_['weight']['fc2']
    weights_dict_['weight']['fc2'] = weights_dict_['weight']['fc2'] * chosen_neurons[0].T

    weights_dict_['weight']['fc3'] = weights_dict_['weight']['fc3'] * chosen_neurons[1].T

    weights_dict_['bias']['fc1'] = chosen_neurons[0].flatten() * weights_dict_['bias']['fc1'].flatten()
    weights_dict_['bias']['fc2'] = chosen_neurons[1].flatten() * weights_dict_['bias']['fc2'].flatten()

    return weights_dict_

def predict_numeric(img_df_row, weights_dict_):

    x = np.dot(weights_dict_['weight']['fc1'], img_df_row.T) + weights_dict_['bias']['fc1']

    x = np.dot(weights_dict_['weight']['fc2'], x) + weights_dict_['bias']['fc2']

    x = np.dot(weights_dict_['weight']['fc3'], x) + weights_dict_['bias']['fc3']

    return x

def print_neurons_shape(weights_dict_):
    print('Layer Number\t|\t Weights Shape\t|\t Bias Shape')
    print('-'*60)
    print('Layer 1  \t|\t', weights_dict_['weight']['fc1'].shape, '\t|\t' , weights_dict_['bias']['fc1'].shape)
    print('Layer 2  \t|\t', weights_dict_['weight']['fc2'].shape, '\t|\t' , weights_dict_['bias']['fc2'].shape)
    print('Layer 3  \t|\t', weights_dict_['weight']['fc3'].shape, '\t|\t' , weights_dict_['bias']['fc3'].shape)
    
def cross_entropy_loss(label, y_hat):
    y = np.zeros(10)
    y[label] = 1
    return -np.sum(y * np.log(y_hat))

print_neurons_shape(weights_dict)

Layer Number	|	 Weights Shape	|	 Bias Shape
------------------------------------------------------------
Layer 1  	|	 (16, 4) 	|	 (16,)
Layer 2  	|	 (8, 16) 	|	 (8,)
Layer 3  	|	 (3, 8) 	|	 (3,)


In [106]:
### Validation of results

my_test_weights = choose_neurons_numeric([list(range(16)), list(range(8))], weights_dict)

preds = np.array([predict_numeric(i, my_test_weights) for i in test_x.values])

counter = 0
for i in range(len(preds)):
    if preds[i].argmax() == test_y.iloc[i]:
        counter += 1

print(counter/150)


def cross_entropy_loss(label, y_hat):

    y = np.zeros_like(y_hat)
    y[np.arange(label.shape[0]), label] = 1 

    exp_preds = np.exp(y_hat)

    softmaxed_preds = exp_preds / exp_preds.sum(axis=1, keepdims=True)

    return -np.sum(y * np.log(softmaxed_preds)) / len(test_y)

print(cross_entropy_loss(test_y, preds)) 

0.9733333333333334
0.06532579124930898


### DoCplex

In [ ]:
from docplex.cp.model import CpoModel as Model

m = Model()

X_fc1 = np.array(m.binary_var_list(16, name='Layer 1 Neurons')).reshape((-1, 1))
X_fc2 = np.array(m.binary_var_list(8, name='Layer 2 Neurons')).reshape((-1, 1))

new_weights_dict = choose_neurons_matmul([X_fc1, X_fc2], weights_dict)

accuracy = 0

preds = np.array([predict_numeric(i, my_test_weights) for i in test_x.values])

loss = cross_entropy_loss(test_y, preds)
    
neurons_used = np.sum(X_fc1) + np.sum(X_fc2) 

m.minimize(loss + (neurons_used*alpha))

In [117]:
# stp = m.create_empty_solution()
# stp[X_fc1] = [1 for i in X_fc1]
# stp[X_fc2] = [1 for i in X_fc2]
# m.set_starting_point(stp)

sol = m.solve(log_output=True)

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 24 variables, 0 constraints
 ! Presolve      : 2 extractables eliminated
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 24.0 (before), 24.0 (after)
 !  . Memory usage      : 300.5 kB (before), 300.5 kB (after)
 ! Using parallel search with 4 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0         24                 -
 + New bound is 0.06532579
      0.06532579        0         24    1            -
 *    0.06532579        0  0.02s        1      (gap is 0.00%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Best objective         : 0.06532579 (optimal - effective tol. is 6.532579e-06)
 ! Best bound             : 0.06532579
 ! -------

In [113]:
[print(sol.get_all_var_solutions()[i]) for i in range(len(sol.get_all_var_solutions()))]

[]

In [148]:
print(sol.get_objective_value())

1223.971001051563
